# Rainfall maps from CML data and merging with radar data
___
<img src="https://hess.copernicus.org/articles/24/2931/2020/hess-24-2931-2020-f07-web.png" alt="drawing" width="800"/>

Accumulated rainfall for a 48 hour show case from 12 to 14 May 2018 for a) RADOLAN-RW, a gauge ajdusted radar product from the German Weaterh Service and b) CML derived rainfall interpolated using inverse distance weighting [Graf et al. 2020](https://hess.copernicus.org/articles/24/2931/2020/).     

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('pycomlink/src')
sys.path.append('poligrain/src')
import xarray as xr
import pycomlink as pycml
import pycomlink.pycomlink.spatial as spatial
import poligrain as plg
import matplotlib.pyplot as plt
import numpy as np
import tqdm

In [ ]:
ds_cml = xr.open_dataset('example_data/openmrg_cml_5min_2h.nc')

In [ ]:
# check mean cml rainfall
ds_cml.R.mean(dim='cml_id').plot()

# Interpolate rainfall maps

In [ ]:
# beware: 
# - the lat and lon coordinates should be converted to UTM (we skip this part)
# - using the link midpoint is an approximation, more advanced teqniques exists. 

<img src="https://gisgeography.com/wp-content/uploads/2016/05/IDW-3Points.png" alt="drawing" width="600"/>


In [ ]:
# define timestep 
time = '2015-07-25 13:25'
# load radar reference data
radar_ref = xr.open_dataset('example_data/openmrg_rad_5min_2h.nc')

# get the center of each CML
ds_cml["lat_center"] = (ds_cml.site_0_lat + ds_cml.site_1_lat) / 2
ds_cml["lon_center"] = (ds_cml.site_0_lon + ds_cml.site_1_lon) / 2

## Inverse distance weighting (IDW)

In [ ]:
# define the properties of the IDW algorithm
idw_interpolator = spatial.interpolator.IdwKdtreeInterpolator(
    nnear=15,  # <------------- number of neighbors
    p=2,  # <------------------ weighting parameter
    exclude_nan=True,  # <----- remove nan values from interpolation
    max_distance=0.4,  # <----- upper bound distance
)

In [ ]:
# interpolate one timestep of rainfall using the lat and lon of the link midpoint. 
R_grid_idw = idw_interpolator(
    x=ds_cml.lon_center,  # <------------------------- input x coordinates
    y=ds_cml.lat_center,  # <------------------------- input y coordinates
    z=ds_cml.R.sel(time=time),  # <----- rainfall values
    xgrid=radar_ref.longitudes.values,  # <--------- target grid x coordinates
    ygrid=radar_ref.latitudes.values,  # <--------- target grid y coordinates
)

In [ ]:
radar_ref['R_grid_idw'] = (('y', 'x'), R_grid_idw)

## Ordinary Kriging

In [ ]:
kriging_interpolator = spatial.interpolator.OrdinaryKrigingInterpolator(
    nlags=6,
    variogram_model='spherical',
    #variogram_parameters=None,
    weight=True,
    n_closest_points=15,
)

In [ ]:
R_grid_kriging = kriging_interpolator(
    x=ds_cml.lon_center.where(~np.isnan(ds_cml.R.sel(time=time)), drop = True), 
    y=ds_cml.lat_center.where(~np.isnan(ds_cml.R.sel(time=time)), drop = True), 
    z=ds_cml.R.sel(time=time).where(~np.isnan(ds_cml.R.sel(time=time)), drop = True),  # <----- rainfall values
    xgrid=radar_ref.longitudes.values,  # <--------- target grid x coordinates
    ygrid=radar_ref.latitudes.values,  # <--------- target grid y coordinates
)

In [ ]:
radar_ref['R_grid_kriging'] = (('y', 'x'), R_grid_kriging)

# Plot IDW and Kriging results

In [ ]:
# such a 2-d array can be visulaized with matplotlibs pcolormesh function
fig, ax = plt.subplots(1, 3, figsize = (4*3, 4))

ax[0].set_title('IDW prediction')
im=ax[0].pcolormesh(radar_ref.longitudes.values, radar_ref.latitudes.values, R_grid_idw,vmin=0,vmax=1)
plg.plot_map.plot_lines(ds_cml, use_lon_lat=True, ax=ax[0]) 
plt.colorbar(im, ax=ax[0])

ax[1].set_title('Kriging prediction')
im = ax[1].pcolormesh(radar_ref.longitudes.values, radar_ref.latitudes.values, radar_ref['R_grid_kriging'],vmin=0,vmax=1)
plg.plot_map.plot_lines(ds_cml, use_lon_lat=True, ax=ax[1])
plt.colorbar(im, ax=ax[1])   

ax[2].set_title('radar reference')
im = ax[2].pcolormesh(
    radar_ref.longitudes.values, 
    radar_ref.latitudes.values, 
    radar_ref.rainfall_amount.sel(time = time, method = 'nearest'),vmin=0,vmax=1
)
plg.plot_map.plot_lines(ds_cml, use_lon_lat=True, ax=ax[2])
plt.colorbar(im, ax=ax[2])
plt.tight_layout();


# Comparing IDW and Kriging to the radar reference

In [ ]:
# crop the radar and estimated rainfall fields so that we compare the interpolated pixels
radar_ref_crop = radar_ref.isel(x = slice(10, 30), y = slice(10, 40))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize= (8, 4))
ax[0].hexbin(
    radar_ref_crop.rainfall_amount.sel(time = time, method = 'nearest').values,
    radar_ref_crop.R_grid_idw.values,
    bins = 'log',
    gridsize = 40,
    extent = (0, 1, 0, 1),
    mincnt=1,
)
ax[0].set_title('IDW')
ax[0].set_xlabel('radar')
ax[0].set_ylabel('cml idw')

ax[1].hexbin(
    radar_ref_crop.rainfall_amount.sel(time = time, method = 'nearest').values,
    radar_ref_crop.R_grid_kriging.values,
    bins = 'log',
    gridsize = 40,
    extent = (0, 1, 0, 1),
    mincnt=1,
)
ax[1].set_title('Kriging');
ax[1].set_xlabel('radar')
ax[1].set_ylabel('cml kriging')
plt.tight_layout();

# Exercise 1
1.1 Interpolate and plot the rainfall sum of the 2.5h example dataset and compare it to the reference
1.2 Remove very short CMLs and interpolate again. Discuss reasons for difference in scatterplot. What other options do we have to improve the  interpolation?

In [ ]:
# 1.1 your solution here



In [ ]:
# one solution for all three exercises
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load solutions/4_1_1_solution.py

In [ ]:
# 1.2 your solution here



In [ ]:
# one solution for all three exercises
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load solutions/4_1_2_solution.py